In [1]:
import src.datasets.tabular
import src.datasets.utils
from src.datasets.tabular import get_dataset_config

## preprocessing for different datasets to vw format
dataset = 'adult'
dataset_kwargs = {'root_dir': 'src/datasets'}
dataset_config = get_dataset_config(dataset, **dataset_kwargs)
dset = src.datasets.utils.get_dataset(dataset_config)
X_tr, y_tr, __, __, __, __ =  dset.get_data()

print(sum(y_tr), len(y_tr))

src.datasets.utils.save_vw_dataset(X_tr, y_tr, dataset, 'src/datasets/adult')


src/datasets
7841 32561


In [8]:
import subprocess, sys, os
VW = 'vw'
ds = 'src/datasets/adult/adult_2.vw.gz'
cmd = [VW, ds, '-b', '24']
# supervised
# cmd += ['--oaa', '2']
# contextual bandit
## --progress 1 means we output all the time loss, otherwise it will only show the time point pow(2,k) 
## param details of different kinds of cb algs are in run_vw_job.py, i.e. the process function.
cmd += ['--cbify', '2', '--bag', '4', '--progress', '1']
output = subprocess.check_output(cmd, stderr=subprocess.STDOUT)
#output = os.system('vw ' + ds + ' -b 24 --oaa 2') 
output = str(output, encoding = 'utf-8')
from src.utils.joint_regret import output_extraction
#the return of output_extraction function is the cummlative loss in terms of the time
cum_loss = output_extraction(output)
cum_loss


array([0.00000000e+00, 1.00000000e+00, 2.00000100e+00, ...,
       7.03300447e+03, 7.03299256e+03, 7.03399003e+03])

In [3]:
!vw src/datasets/adult/adult_2.vw.gz 24 --oaa 2

[warning] Multiple data files passed as positional parameters, only the first one will be read and the rest will be ignored.
using no cache
Reading datafile = src/datasets/adult/adult_2.vw.gz
num sources = 1
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
Enabled reductions: gd, scorer-identity, oaa
Input label = MULTICLASS
Output pred = MULTICLASS
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features
0.000000 0.000000            1            1.0              1              1       11
0.000000 0.000000            2            2.0              1              1       11
0.000000 0.000000            4            4.0              1              1       12
[info] label 2 found -- labels are now considered 1-indexed.
0.000000 0.000000            8            8.0              2              2       11
0.437500 0.875000           16           16.0            

In [ ]:
!vw src/datasets/german/german_2.vw.gz -b 24 --cbify 2 --regcbopt

In [9]:
## the falcon procedure, for the paper
## https://arxiv.org/pdf/2003.12699.pdf
from src.models.cb.FALCON import FALCON, FALCON_ldf, FALCON_price
#from split_gz_csv import ds_files_csv, ds_files
#from split import sample_custom_pmf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
#from glob_param import *
from src.run_vw_job import process
#from split import *
import re
rgx = re.compile('^average loss = (.*)$', flags=re.M)
#show the regret performance in each algorithm

csv_path = 'src/datasets/adult/adult_2.csv'
# def alg_performance_regret(DS_DIR, did, alg_param, did_type):
loss_param = []
    # if did_type == 'openml':
    #     sample_falcon = FALCON(csv_path, 95, 1)
    # elif did_type == 'ms':
    #     sample_falcon = FALCON_ldf(csv_path, 105, 1, 10)
    # elif did_type == 'yahoo':
    #     sample_falcon = FALCON_ldf(csv_path, 105, 1, 6, 'ridge')
    # elif did_type == 'loan':
    #     if did == 0:
    #         sample_falcon = FALCON_price(csv_path, 10, 1, 15)
    #     elif did == 1:
    #         sample_falcon = FALCON_price(csv_path, 10, 1, 10)
#gamma_param is a hyperparam in FALCON alg.
sample_falcon = FALCON(csv_path, 95, 1)
    
sample_falcon.learn_schedule()

#cummulative loss for FALCON alg.
sample_falcon.loss_all
        
    # gz_path = ds_files(DS_DIR)[did]
    # for index in range(len(alg_param)):
    #     pv_loss, output = process(gz_path, alg_param[index], None, True, did_type)
    #     loss_param.append(list(output_extraction(output)))
    # return loss_param

3


array([   0.,    0.,    0., ..., 6008., 6008., 6008.])

In [6]:
loss_param

[array([   0.,    0.,    0., ..., 5990., 5990., 5990.])]

In [7]:
output_all = output.split('\n')
res = []
for i in range(len(output_all)):
    if len(output_all[i].split(' ')) >= 50:
        res.append(float(output_all[i].split(' ')[0]))
        #print(len(output_all[i].split(' ')))
    #print(len(output_all[i].split(' ')))
len(res)

32561